In [1]:
!pip install lxml
!pip install selenium
!pip install cssselect
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [1]:
import pandas as pd
import numpy as np

from lxml import html
from lxml.cssselect import CSSSelector

# Create the web driver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [2]:
# Load the site on the web driver
url = "https://axie.zone/leaderboard"
driver.get(url)
driver.implicitly_wait(10)

# List to store links pointing to the player pages
player_link = []

try:
  # Find the anchor tags in the leaderboard table
  links_selector = ".leaderboard_table a"
  anchors = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, links_selector)))

  # Extract the href from the anchors found in the table
  for anchor in anchors:
    anchor_href = anchor.get_attribute("href")
    player_link.append(anchor_href)

  print("Retrieved", len(player_link), "URLs")
except TimeoutException:
  print("Element not properly loaded")

Retrieved 100 URLs


In [3]:
import re

# Used to extract the Axie parts from the user page
def get_parts(players):
  # List to store the scraped data
  axie_data = []

  # Counter that will be used to track the progress of the function
  progress = 0

  # Regex pattern to parse the parts
  p = re.compile('(.*): (.*) (\[.*\])')

  # Iterate through each of the player in the leaderboards
  for player in players:
    progress += 1
    try:
      # Load the player page on the driver
      driver.get(player)
      driver.implicitly_wait(10)

      # Wait till the Axie info container loads
      _ = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#last_used_team_container > div")))

      # Get the element that wraps the last used team container
      # This can be refactored to include the most used team of the player as well
      axie_container = WebDriverWait(driver, 0).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#last_used_team_container")))

      # Convert the selenium webelement into a LXML tree
      axie_html = axie_container[0].get_attribute('innerHTML')
      axie_tree = html.fromstring(axie_html)

      # Get the Axie containers
      info_containers = axie_tree.cssselect(".search_result_wrapper")

      for info_container in info_containers:
        axie_parts = {}

        # Get the address of the owner
        axie_parts["User_Addr"] = player.replace("https://axie.zone/profile?ron_addr=", "")

        # Get the Axie class
        axie_parts["Class"] = info_container.cssselect(".search_result")[0].get('class').replace("search_result ", "").capitalize()

        # Scrapes the parts from the Axie container
        parts = info_container.cssselect(".purity_part > div")

        for part in parts:
          m = p.match(part.get('title'))
          axie_parts[m.group(1)] = m.group(2)

        # Add the scraped parts to the main list of data
        axie_data.append(axie_parts)

      # To keep track and now that our script is working, we would print a message
      # containing a summary of the scraping's progress
      print("(", progress, "/ 100 ) Retrieved")
    except Exception as e:
      # We might also encounter an error where our collection script fails,
      # in the rare case where this would happen we would need to manually
      # revisit what went wrong and fix it.
      print("(", progress, "/ 100 ) Error occured while retrieving", player)

  return axie_data
  
# Start the scrapping process
res = get_parts(player_link)

( 1 / 100 ) Retrieved
( 2 / 100 ) Retrieved
( 3 / 100 ) Retrieved
( 4 / 100 ) Retrieved
( 5 / 100 ) Retrieved
( 6 / 100 ) Retrieved
( 7 / 100 ) Retrieved
( 8 / 100 ) Retrieved
( 9 / 100 ) Retrieved
( 10 / 100 ) Retrieved
( 11 / 100 ) Retrieved
( 12 / 100 ) Retrieved
( 13 / 100 ) Retrieved
( 14 / 100 ) Retrieved
( 15 / 100 ) Retrieved
( 16 / 100 ) Retrieved
( 17 / 100 ) Retrieved
( 18 / 100 ) Retrieved
( 19 / 100 ) Retrieved
( 20 / 100 ) Retrieved
( 21 / 100 ) Retrieved
( 22 / 100 ) Retrieved
( 23 / 100 ) Retrieved
( 24 / 100 ) Retrieved
( 25 / 100 ) Retrieved
( 26 / 100 ) Retrieved
( 27 / 100 ) Retrieved
( 28 / 100 ) Retrieved
( 29 / 100 ) Retrieved
( 30 / 100 ) Retrieved
( 31 / 100 ) Retrieved
( 32 / 100 ) Retrieved
( 33 / 100 ) Retrieved
( 34 / 100 ) Retrieved
( 35 / 100 ) Retrieved
( 36 / 100 ) Retrieved
( 37 / 100 ) Retrieved
( 38 / 100 ) Retrieved
( 39 / 100 ) Retrieved
( 40 / 100 ) Retrieved
( 41 / 100 ) Retrieved
( 42 / 100 ) Retrieved
( 43 / 100 ) Retrieved
( 44 / 100 ) Retriev

In [4]:
# Convert the python list into a dataframe
df_res = pd.DataFrame(res)
df_res.head()

,User_Addr,Class,Eyes,Ears,Back,Mouth,Horn,Tail
0,0x6f08189713d7c88b6f69b671c9ac5f0081cc9ba5,Aquatic,Clear,Leafy,Goldfish,Piranha,Cuckoo,Koi
1,0x6f08189713d7c88b6f69b671c9ac5f0081cc9ba5,Beast,Little Owl,Risky Bird,Ronin,Nut Cracker,Imp,Nut Cracker
2,0x6f08189713d7c88b6f69b671c9ac5f0081cc9ba5,Plant,Cucumber Slice,Lotus,Pumpkin,Serious,Leaf Bug,Cattail
3,0x7f7cdcb632d1641378086ce89ab4190b61677805,Bug,Neo,Ear Breathing,Garish Worm,Pincer,Parasite,Fish Snack
4,0x7f7cdcb632d1641378086ce89ab4190b61677805,Plant,Papi,Hollow,Garish Worm,Serious,Leaf Bug,Yam


In [6]:
# Export the dataframe into a csv file
df_res.to_csv("leaderboard.csv", index=False)